In [24]:
import torch
import numpy as np
import pandas as pd

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [11]:
outfile = open("../models/mimic-iii/raw_text_questions.txt", 'r')
# for entry in openMessages:
corpus = outfile.read().split("\n")

corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [23]:
ce = corpus_embeddings.cpu().numpy()
np.savetxt("../models/mimic-iii/corpus.csv", ce, delimiter=',')

In [38]:
query = "what percentile is the value of 1.1 in a creatinine lab test amongst patients of the same age as patient 21814 on the current hospital visit?"
top_k = min(10, len(corpus))

query_embedding = model.encode(query, convert_to_tensor=True).to('cpu').to(torch.float32)

corpus_embeddings = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 10 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    print(corpus[idx], "(Score: {:.4f})".format(score))

"""
# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
    print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
"""





Query: what percentile is the value of 1.1 in a creatinine lab test amongst patients of the same age as patient 21814 on the current hospital visit?

Top 10 most similar sentences in corpus:
what was the last value on the sodium lab test of patient 42956? (Score: 0.9698)
hmm.. what was patient 49295's last ph until 6 months ago? (Score: 0.9693)
what was the percentile of 18.9 in the rdw lab test with the same age of patient 7112 during this hospital encounter? (Score: 0.9636)
what's patient 51675's maximum value of monocytes until 40 months ago? (Score: 0.9632)
what is the percentile hematocrit value of 32.7 among patients of the same age as patient 41393 when they came to the hospital last time? (Score: 0.9619)
hmm.. what was patient 97733's last mchc since 12/2105? (Score: 0.9612)
until 03/2104, what was the average mcv value for patient 69289? (Score: 0.9611)
what is the percentile value of creatinine 0.3 among patients of the same age as patient 88401 on the first hospital visi

'\n# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk\nhits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)\nhits = hits[0]      #Get the hits for the first query\nfor hit in hits:\n    print(corpus[hit[\'corpus_id\']], "(Score: {:.4f})".format(hit[\'score\']))\n'

In [40]:
top_results

torch.return_types.topk(
values=tensor([0.9698, 0.9693, 0.9636, 0.9632, 0.9619, 0.9612, 0.9611, 0.9609, 0.9608,
        0.9597], dtype=torch.float32),
indices=tensor([2373, 2368,  914, 2503,  913, 2310, 2514,  872,  616,  894]))